# GASP (Generalized Algorithm for Signed Graph Partitioning)

Use the `elf.segmentation` module for boundary based segmentation with the GASP algorithm: [A Generalized Framework for Agglomerative Clustering of Signed Graphs applied to Instance Segmentation](https://arxiv.org/abs/1906.11713).
Here we use some example data taken from the [ISBI 2012 EM Segmentation challenge](http://brainiac2.mit.edu/isbi_challenge/home).
You can obtain this data [here](https://hcicloud.iwr.uni-heidelberg.de/index.php/s/6LuE7nxBN3EFRtL).

GASP is a theoretical framework that generalizes
simple and fast algorithms for hierarchical agglomerative
clustering to weighted graphs with both attractive and repulsive interactions between the nodes.

GASP can be used to cluster a weighted graph or can operate directly on pixel affinity maps. In the latter case, it produces a segmentation by partitioning the grid graph, taking into account long range pixel connections.


## Preparation

In [1]:
import numpy as np

# import napari for data visualisation
import napari

# Import function to download the example data
from elf.segmentation.utils import load_mutex_watershed_problem

# import the segmentation functionality from elf
from elf.segmentation import GaspFromAffinities
from elf.segmentation import run_GASP

# Import an utility function from nifty that we will need to generate a toy graph:
from nifty.graph import UndirectedGraph

# import the open_file function from elf, which supports opening files
# in hdf5, zarr, n5 or knossos file format
from elf.io import open_file

In [7]:
# Download some example data and affinities:
prefix = "isbi-data-"
data_path = f"{prefix}test.h5"
affs, offsets = load_mutex_watershed_problem(prefix=prefix)
affs = 1. - affs
with open_file(data_path, 'r') as f:
    # load the raw data in addition
    raw = f['raw'][:]

## Segment an image using GASP

In [8]:
# We first need to decide which linkage criteria to use and whether
# to use cannot-link constraints in the agglomeration. The
# avbailable linkage criteria are:
#     - `mean`, `average`, `avg` (average hierarchical clustering)
#     - `max`, `single_linkage` (single hierarchical clustering)
#     - `min`, `complete_linkage` (complete hierarchical clustering)
#     - `mutex_watershed`, `abs_max` (mutex watershed algorithm)
#     - `sum` (GAEC algorithm)

# Here, as an example, we use average hierarchical clustering without
# cannot-link constraints:
run_GASP_kwargs = {'linkage_criteria': 'average',
                   'add_cannot_link_constraints': False}

# Run the algorithm:
gasp_instance = GaspFromAffinities(offsets,
                                   run_GASP_kwargs=run_GASP_kwargs)
# To speed-up computations, here we use only part of the example data:
segmentation, runtime = gasp_instance(affs[:,:10])


In [9]:
# Visualize the result:
viewer = napari.Viewer()
viewer.add_image(raw[:10], name='raw')
viewer.add_image(affs[:,:10], name='affinities')
viewer.add_labels(segmentation, name='gasp-avg-segmentation')

<Labels layer 'gasp-avg-segmentation' at 0x7fe820e23850>

### Option to use a superpixel generator
Instead of segmenting an image starting from the pixel grid-graph, we can also first run an algorithm to generate superpixels and then use GASP on the generated graph.

Here, as example, we will use a distance transform watershed algorithm to generate superpixels.


In [11]:
from elf.segmentation.watershed import WatershedOnDistanceTransformFromAffinities

# Here we define the superpixel generator.
# You can also define your own generator. It should expect affinities and it should return a segmentation as output.
superpixel_gen = WatershedOnDistanceTransformFromAffinities(offsets,
                                                            threshold=0.4,
                                                            sigma_seeds=0.1,
                                                            min_size=20,
                                                            stacked_2d=True,
                                                            used_offsets=[1, 2],
                                                            )


In [12]:
# Define another GASP instance, this time adding the superpixel generator:
sp_gasp_instance = GaspFromAffinities(offsets,
                                      superpixel_generator=superpixel_gen,
                                      run_GASP_kwargs=run_GASP_kwargs)
gasp_segmentation_sp, runtime_sp = sp_gasp_instance(affs)

In [13]:
# For visualization, let's compute again the generated superpixels and save them into
# an array:
superpixel_segm = superpixel_gen(affs)

# Visualize the new segmentation in the viewer:
viewer = napari.Viewer()
viewer.add_image(raw, name='raw')
viewer.add_image(affs, name='affinities')
viewer.add_labels(superpixel_segm, name='superpixels-segmentation')
viewer.add_labels(gasp_segmentation_sp, name='gasp-segmentation-sp')

<Labels layer 'gasp-segmentation-sp' at 0x7fe82044d280>

## Clustering a weighted graph using GASP

GASP can also be used to cluster a generic graph with both positive and negative weights.

In [14]:
# --------------------------
# First, we generate a toy undirected graph with signed weights:
# --------------------------
nb_nodes, nb_edges = 7000, 10000
graph = UndirectedGraph(nb_nodes)
# Generate some random connections between nodes (avoiding self-loop):
random_edges = np.random.randint(0, nb_nodes - 1, size=(nb_edges, 2))
self_loops = np.argwhere(random_edges[:,0] == random_edges[:,1])
random_edges = np.delete(random_edges, self_loops, axis=0)
# Add connections to the graph:
graph.insertEdges(random_edges)
# Now, let's sample some random (signed) weights:
random_signed_weights = np.random.uniform(-1., 1., size=graph.numberOfEdges)

In [17]:
# And finally, we run GASP on the toy graph:
# the output of the GASP function is an array of node labels, such that all nodes in the
# same cluster are assigned to a distinct label.
node_labels_gasp_clustering, runtime_gasp_graph = run_GASP(graph,
                                      random_signed_weights,
                                      linkage_criteria='average',
                                      add_cannot_link_constraints=False,
                                      verbose=False,
                                      print_every=100)

(7000,)
